In [6]:
class State:
    def __init__(self,row,col,grid):
        self.row=row
        self.col=col
        self.grid=grid
        self.n=len(grid)
    
    def goalTest(self):
        return self.row==self.n-1 and self.col==self.n-1
    
    def moveGen(self):
        directions=[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
        children=[]
        for dr,dc in directions:
            new_row,new_col=self.row+dr,self.col+dc
            if 0<=new_row<self.n and 0<=new_col<self.n:
                if(self.grid[new_row][new_col]==0):
                    children.append(State(new_row,new_col,self.grid))
        return children
    
    def __eq__(self, other):
        return self.row==other.row and self.col==other.col
    
    def __hash__(self):
        return hash((self.row,self.col))
    
    def __str__(self):
        return f" ({self.row} , {self.col}) "
    
    def h(state):
        idx = state.n - 1
        return abs(state.row - idx) + abs(state.col - idx)
    
    def stepCost(self, other):
        return 1
    
#FUNCTION RELATED TO A* AND BEST_FIRST_SEARCH

def is_valid_start_and_goal(grid):
    n = len(grid)
    if grid[0][0] != 0:
        return False
    if grid[n-1][n-1] != 0:
        return False
    return True

In [5]:
#A_STAR FUNCTIONS

def reconstructPath_Astar(node, parent_map):
  path = [node]
  parent = parent_map[node]
  while parent is not None:
    path.append(parent)
    parent = parent_map[parent]
  path.reverse()
  return path

def propagateImprovement(M, parent_map, g, f, CLOSED):
  for X in M.moveGen():
    k = M.stepCost(X)
    new_g = g[M] + k
    if new_g < g.get(X, float('inf')):
      parent_map[X] = M
      g[X] = new_g
      f[X] = g[X] + X.h()
      if X in CLOSED:
        propagateImprovement(X, parent_map, g, f, CLOSED)

def a_star(start):
  if not is_valid_start_and_goal(start.grid):
       return []
  
  OPEN = [start]
  CLOSED = []

  parent_map = {start:None}
  g = {start:0}
  f = {start: g[start] + start.h()}

  while OPEN:
    N = min(OPEN, key = lambda state: f.get(state))
    OPEN.remove(N)

    if N.goalTest():
    #   print("Goal found")
      path = reconstructPath_Astar(N, parent_map)
      return path

    else:
      CLOSED.append(N)

      for M in N.moveGen():
        k = N.stepCost(M)
        new_g = g[N] + k

        if new_g < g.get(M, float('inf')):
          parent_map[M] = N
          g[M] = new_g
          f[M] = g[M] + M.h()

          if M in CLOSED:
            propagateImprovement()
          elif M not in OPEN:
            OPEN.append(M)

  return []

In [ ]:
#BEST_FIRST_SEARCH FUNCTIONS
def removeSeen(children, open_list, closed_list):
    open_nodes = [node for node, _, _ in open_list]
    closed_nodes = [node for node, _, _ in closed_list]
    return [c for c in children if c not in open_nodes and c not in closed_nodes]

def reconstructpathbfs(node_pair, closed):
    parent_map = {node: parent for node, parent, _ in closed}
    node, parent = node_pair
    path = [node]
    while parent:
        path.append(parent)
        parent = parent_map[parent]
    return path[::-1]


#BEST_FIRST_SEARCH
def best_first_search(start):
    if not is_valid_start_and_goal(start.grid):
       return []
       
    OPEN = [(start, None,start.h())]
    CLOSED = []

    while OPEN:
        node, parent, hval = OPEN.pop(0)
        CLOSED.append((node, parent, hval))
        if node.goalTest():
            return reconstructpathbfs((node, parent), CLOSED)

        children = node.moveGen()
        new_nodes = removeSeen(children, OPEN, CLOSED)
        for m in new_nodes:
            OPEN.append((m, node, m.h()))
        OPEN.sort(key=lambda x: x[2])

    return []


In [7]:
grid = [[0,0,0],[1,1,0],[1,1,0]]
# grid = [[1,0,0],[1,1,0],[1,1,0]]
# grid=[[0,1],[1,0]]
start_state = State(0, 0, grid)
path1 = best_first_search(start_state)
   
if path1:
    count=0
    for p in path1:
        count+=1
    print("BEST FIRST SEARCH -> Path length:",count,", Path:","[" + ", ".join(str(p) for p in path1) + "]")
else:
   print("BEST FIRST SEARCH -> Path length:",-1)
   
path2=a_star(start_state)

if path2:
    c=0
    for p in path2:
        c+=1
    print("A* SEARCH -> Path length:",c,", Path:","[" + ", ".join(str(p) for p in path2) + "]")
else:
   print("A* SEARCH -> Path length:",-1)

BEST FIRST SEARCH -> Path length: 4 , Path: [ (0 , 0) ,  (0 , 1) ,  (1 , 2) ,  (2 , 2) ]
A* SEARCH -> Path length: 4 , Path: [ (0 , 0) ,  (0 , 1) ,  (1 , 2) ,  (2 , 2) ]
